# 5.3 RAG 기반의 챗봇 만들기
- 라이브러리: langchain, openai, PyPDF2, sentence-transformers
- 언어모델: gpt-3.5-turbo
- 임베딩모델: all_MiniLM-L6-v2
- 벡터 데이터베이스: 파이스(FAISS)
  

## 코드 흐름
> 사용자가 PDF 파일을 업로드하면 그것을 청크로 분할하고 임베딩 처리  
> 그리고 LLM 한테 해당 파일을 요약해 달라고 쿼리를 던지는 흐름

In [2]:
# !pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [4]:
import os
from PyPDF2 import PdfReader
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

# 전처리
def process_text(text):
    #CharacterTextSplitter를 사용하여 텍스트를 청크로 분할
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size =1000,
        chunk_overlap=200,
        length_function = len
    )
    chunks = text_splitter.split_text(text)

    # 임베딩 처리(벡터 변환), 임베딩은 HuggingFacsEmbeddings 모델 사용
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    documents = FAISS.from_texts(chunks, embeddings)
    return documents


def main(): #streamlit을 이용한 웹사이트 생성
    st.title("📜PDF 요약하기")
    st.divider()
    try:
        # os.environ["OPENAI_API_KEY"] = ""
    except ValueError as e:
        st.error(str(e))
        return

    pdf = st.file_uploader("PDF파일을 업로드해 주세요", type='pdf')

    if pdf is not None:
        pdf_reader = PdfReader(pdf)
        text = "" # 텍스트 변수에 PDF 내용을 저장
        for page in pdf_reader.pages:
            text += page.extract_text()

        documents = process_text(text)
        query = "업로드된 PDF 파일을 내용을 약 3~5문장으로 요약해주세요." # LLM에 PDF파일 요약 요청

        if query:
            docs = documents.similarity_search(query)
            llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0.1)
            chain = load_qa_chain(llm, chain_type = "stuff")
    
            with get_openai_callback() as cost:
                response = chain.run(input_documents=docs, question=query)
                print(cost)
            st.subheader('--요약결과--')
            st.write(response)

# 파이썬 파일이 사용자에 의해 직접 실행되면 main() 함수를 실행하라
if __name__ == "__main__":
    main()